<a href="https://colab.research.google.com/github/mch60131242/Home/blob/main/%EA%B5%90%EC%B0%A8_%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import numpy as np
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [20]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [21]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data,target, test_size=0.2, random_state=42)
sub_input, val_input, sub_target,val_target = train_test_split(train_input,train_target, test_size=0.2, random_state=42)

In [22]:
print(train_input.shape,test_input.shape, sub_input.shape,val_input.shape) #data =train_input+test_input train_input =sub_input+val_input

(5197, 3) (1300, 3) (4157, 3) (1040, 3)


In [23]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))

0.9971133028626413
0.8576923076923076


In [24]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt,train_input,train_target)#cv의 default = 5
display(scores)# 학습시간,검증시간(검증세트로돌린시간),테스트점수



{'fit_time': array([0.0100739 , 0.00882149, 0.01038265, 0.01048684, 0.00983334]),
 'score_time': array([0.00142503, 0.00177574, 0.00165534, 0.00184751, 0.00142264]),
 'test_score': array([0.86538462, 0.84423077, 0.87391723, 0.85081809, 0.84119346])}

In [25]:
print(np.mean(scores['test_score']))

0.8551088324572443


In [26]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt,train_input,train_target, cv =StratifiedKFold())
print(np.mean(scores['test_score']))

splitter =StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt,train_input,train_target, cv =splitter)
print(np.mean(scores['test_score']))

0.856646738728067
0.8574195938935825


In [27]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001,0.0002,0.0003,0.0004,0.0005]}  ##파라미터 지정
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) #n_jobs 사용할 cpu 개수 선택 -1은 전부라는뜻
gs.fit(train_input,train_target)
dt= gs.best_estimator_
print(dt.score(train_input,train_target))
print(gs.best_params_)
print(gs.cv_results_['mean_test_score'])
best_index=np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


In [28]:
##특성은 데이터의 타겟을 설명할 수 있는 특징들(열의 이름이 주로 특징이 됨)
##훈련

In [29]:
## 분류의 평가
## 데이터 전처리  표준점수 or 데이터 스케일링 (기준점수-평균점수)/표준편차
##numpy broadcasting <- 차원이 맞지 않는 두 array의 계산 
##회귀(예측)  
## 분류는 정확도 라는 기준 회귀는 R square또는 결정계수 ,
## 과대적합(학습을 너무 잘함) ##과소적합(학습을 너무 못함)
## 과대적합은(학습점수가 테스트 점수보다 높은 상태) 과소적합(학습점수 테스트 점수가 낮은 상태)
##선형회귀-계수/가중치,모델 파라미터(학습 변수) y(타겟) = a(계수)x +b(절편) a,b가 가중치
#모델 파라미터는 모델 자체적으로 학습하는 변수 이기에 사용자가 개입할 여지가 없음.


In [30]:
## 다항회귀는 선형회귀의 패턴의 다양화로써 규칙을 늘려주는 행위
## 다중회귀,  특성공학,
##데이터 전처리와 특성 공학은 가장 중요한 part중 하나
##특성공학  :data selection,data extraction 분류를 위해서는 어떤 기준을 선택하고 확장하고 추출 할 것인가?-<data extraction 어떠한 특성을 기준으로 할 것인가?,데이터를 만드는 기준<-data selection
## ex 배민의 배달 <- 배달 예상 시간을 예측하기 위해선 어떠한 데이터를 모아야 하는가? <-data selection  모아놓은 데이터 중에서 어떤것이 활용하기에 좋은가<- data extraction
## 특성공학 = feature engineering
##규제 L1(Lasso) L2(Ridge) <-우리가 찾는 계수(가중치)에 임의 값을 곱해주고 늘려줌으로써 규제의 강도를 조절 (규제는 데이터가 학습을 너무 잘하는것을 막기 위해)
##로지스틱 회귀 0~1사이로 다중 분류 시그모이드 함수 소프트 맥스 함수 이진 분류와 다중 분류
## 확률적 경사하강법 /손실함수/에포크 손실함수는 평가하는 기준   오차를 줄이는게 목표
## 손실함수인 평가지표는 미분 가능해야한다.
## 확률적 경사 하강법은 딥러닝에서 많이 배우게 된다
#결정트리- 노드를 놔눠 간다 루트,리프, 불순도, 불순도와 정보이득 결정트리는 기본적으로 과대적합이 생길 수 밖에없는 형태이기 떄문에 가지치기(규제)를 통해서 과대적합 문제를 해결한다
#특성중요도를 결정트리에서 자동으로 계산한다. # 매번 테스트 세트를 활용하는것은 결국 학습데이터를 테스트 데이터에 맞추는 형태로 진행 됨
#따라서 우리는 학습 데이터를 검증데이터를 나눠서 사용하게 된다 학습 set 검증 set 테스트 set
# cross validation은 학습데이터를 스스로 5조각(기본)으로 나눠서 학습 데이터 검증 데이터로 나눠준다 
# Grid Search 하이퍼 파라미터 튜닝을 위한것으로 탐색과 교차 검증을 한번에 해주는 클래스로 최적의 하이퍼 파라미터를 찾는데 유용하다.
#

In [33]:
params = {'min_impurity_decrease': np.arange(0.0001,0.001,0.0001),  ## 9개
          'max_depth': range(5,20,1),  ## 15개
          'min_samples_split': range(2,100,10)}  ## 10개
          # 총 1350개의 교차 검증 x 5(cv의 검증 데이터 교차 횟수)= 6750개의 모델생성




In [35]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params,n_jobs =-1)
gs.fit(train_input,train_target)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
print(gs.best_params_)

In [38]:
display(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731

In [40]:
from scipy.stats import uniform,randint #uniform은 실수 randint는 정수를 랜덤으로 뽑아줌
rgen = randint(0,10)
rgen.rvs(10) ## 범위 안에서 몇개
np.unique(rgen.rvs(1000),return_counts=True) ## 0이 105개 1이 107개 ....9가 104개

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([105, 107,  97,  78, 111, 104,  95, 111,  88, 104]))

In [44]:
ugen =uniform(0,0.1)
ugen.rvs(10)

array([0.08891837, 0.04309906, 0.09327303, 0.04365235, 0.09975236,
       0.03672498, 0.06829684, 0.00786106, 0.03149387, 0.09502474])

In [45]:
params = {'min_impurity_decrease': uniform(0.0001,0.001), 
          'max_depth': randint(20,50),  
          'min_samples_split': randint(2,25),
          'min_samples_leaf': randint(1,25)}  


In [47]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, n_iter=100, random_state=42) ## n_iter <- 총 100번을 샘플링하여 교차 검증을 수행하라
gs.fit(train_input,train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7eff8a6fbc10>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7eff8847ef10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7eff884890a0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7eff8847ed60>},
                   random_state=42)

In [48]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [49]:
dt = gs.best_estimator_
print(dt.score(test_input,test_target))

0.86


In [52]:
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42,splitter='random'), params, n_jobs=-1, n_iter=100, random_state=42)
gs.fit(train_input,train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
